In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgbm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


## In this notebook I want to share my main tree models that helped for adding diversity for my final stacking.

1 - First model is my main best single model.

2 - Second model is a 2-stage lgbm. As a first thing I turn target column into a binary column and run 1st model to predict the probability of being class 1 for each row. In 2nd stage models, I only use training data with class 1 and get real target predictions for validation and test set. Then doing the same by only using training data with class 0. At then end I calcualte final prediction as like this:

final_prediction = probability_of_being_0 * (predictions_from_training_0) + probability_of_being_1 * (predictions_from_training_1)

3 - As a third model, I tried to apply dae. Even though I failed with it (given the 1st place's solution :)), it added diverstiy to my final blending in the end. First I run a simple MLP model with a linear output and then run it for a couple epochs. Later on, I used the MLP output for augmenting the original training data during my CV.

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
cont_features = [col for col in train.columns if col.startswith("cont")]
len(cont_features)

## 1st type of modeling

My plain best lgbm model. You can get 6952-6953 cv score with one seed modeling and 0.001 learning rate. But since it takes more time, I prefer to run a fast model with a higher learning rate for a couple of seeds and then average their predictions. Here I also used init_model parameter of lightgbm model to squeeze more from the data.[You can also check this notebook](https://www.kaggle.com/fatihozturk/lgbm-model-initialisation-trick). 

In [ ]:
y = train["target"]
kf = KFold(n_splits=5, shuffle=True, random_state=1)
oof = np.zeros(len(train))
score_list = []
fold = 1
test_preds = []
seed_list = [None,2,3]# Use more. Original list: [None,2,3,4,5]


for train_index, test_index in kf.split(train):
    X_train, X_val = train.iloc[train_index], train.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
    

    X_train = X_train.abs() # Taking aboslute was also a bit improving.

    y_pred_list = []
    for seed in seed_list:
        dtrain = lgbm.Dataset(X_train[cont_features], y_train)
        dvalid = lgbm.Dataset(X_val[cont_features], y_val)
        print(seed)
        params = {"objective": "regression",
              "metric": "rmse",
              "verbosity": -1,
              "boosting_type": "gbdt",
              "feature_fraction":0.5,
              "num_leaves": 250,
              "lambda_l1":7,
              "lambda_l2":2,
              "learning_rate":0.01,
              'min_child_samples': 35,
              "bagging_fraction":0.75,
              "bagging_freq":1,
             }
        params["seed"] = seed
        model = lgbm.train(params,
                        dtrain,
                        valid_sets=[dtrain, dvalid],
                        verbose_eval=100,
                        num_boost_round=100000,
                        early_stopping_rounds=100
                    )

        dtrain = lgbm.Dataset(X_train[cont_features], y_train)
        dvalid = lgbm.Dataset(X_val[cont_features], y_val)
        params = {"objective": "regression",
                  "metric": "rmse",
                  "verbosity": -1,
                  "boosting_type": "gbdt",
                  "feature_fraction":0.5,
                  "num_leaves": 350,
                  "lambda_l1":7,
                  "lambda_l2":1,
                  "learning_rate":0.003,
                  'min_child_samples': 35,
                  "bagging_fraction":0.8,
                  "bagging_freq":1,
                 }
        
        params["seed"] = seed
        model = lgbm.train(params,
                            dtrain,
                            valid_sets=[dtrain, dvalid],
                            verbose_eval=100,
                            num_boost_round=100000,
                            early_stopping_rounds=200,
                           init_model = model
                        )

    
    
        y_pred_list.append(model.predict(X_val[cont_features]))
        print(np.sqrt(mean_squared_error(y_val,   np.mean(y_pred_list,axis=0)       )))
        test_preds.append(model.predict(test[cont_features]))
        
    
   
    
    oof[test_index] = np.mean(y_pred_list,axis=0)    
    score = np.sqrt(mean_squared_error(y_val, oof[test_index]))
    score_list.append(score)
    print(f"RMSE Fold-{fold} : {score}")
    fold+=1

np.mean(score_list)

In [ ]:
print(score_list)
np.mean(score_list)

In [ ]:
train["1_preds"] = oof
test["1_preds"] = np.mean(test_preds,axis=0)

## 2nd type of modeling
Second model is a 2-stage modeling. As a first thing I turn target column into a binary column and run 1st model to predict the probability of being class_1 for each row. In 2nd stage models, I do separete modeling again. I only use training data with class 1 and get real target predictions for validation and test set. Then doing the same by only using training data with class 0. At then end I calcualte final prediction as:

final_prediction = probability_of_being_0 * (predictions_from_training_0) + probability_of_being_1 * (predictions_from_training_1)

With this method, you can just change follwoing "**threshold**" parameter and get more results to use for final stacking. **This is what I did to increase diversity.**

In [ ]:
threshold = 8
train["target_class"] = train["target"].apply(lambda x: 1 if x > threshold  else 0)
train.groupby("target_class")["target"].mean()

In [ ]:
train[train.target_class == 0].target.hist(bins=100)
train[train.target_class == 1].target.hist(bins=100)

In [ ]:
#X = X.abs()
from sklearn.metrics import roc_auc_score

y = train["target_class"]
kf = KFold(n_splits=5, shuffle=True, random_state=1)
oof = np.zeros(len(train))
score_list = []
fold = 1
test_preds = []

for train_index, test_index in kf.split(train):
    X_train, X_val = train.iloc[train_index], train.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
   

    #X_train = X_train.abs()
    dtrain_0 = lgbm.Dataset(X_train[cont_features], y_train)
    dvalid = lgbm.Dataset(X_val[cont_features], y_val)
    
    params = {"objective": "binary",
              "metric": "auc",
              "verbosity": -1,
              "boosting_type": "gbdt",
              "feature_fraction":0.5,
              "num_leaves": 200,
              "lambda_l1":7,
              "lambda_l2":0,
              "learning_rate":0.01,
              'min_child_samples': 0,
              "bagging_fraction":0.75,
              "bagging_freq":1,
             }
    model_0 = lgbm.train(params,
                    dtrain_0,
                    valid_sets=[dtrain_0, dvalid],
                    verbose_eval=100,
                    num_boost_round=100000,
                    early_stopping_rounds=100
                )
    
    y_pred_0 = model_0.predict(X_val[cont_features])    
    score = roc_auc_score(y_val,y_pred_0)

    print(f"AUC Fold-{fold} : {score}")
    
    oof[test_index] = y_pred_0
    
    
    score_list.append(score)
    test_preds.append(model_0.predict(test[cont_features]))
    fold+=1
    
np.mean(score_list)

In [ ]:
# Here I get classification predictions for both train and test data.
train["class_preds"] = oof
test["class_preds"] = np.mean(test_preds,axis=0)
roc_auc_score(train["target_class"],train["class_preds"])

In [ ]:
#X = X.abs()
y = train["target"]
kf = KFold(n_splits=5, shuffle=True, random_state=1)
oof_2 = np.zeros(len(train))
score_list = []
fold = 1
test_preds_2 = []

for train_index, test_index in kf.split(train):
    X_train, X_val = train.iloc[train_index], train.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
    

    # First model with only X_train having class = 0
    #X_train = X_train.abs()
    X_train_0 = X_train[X_train.target_class==0]
    y_train_0 = y_train[X_train.target_class==0]
    
    X_train_1 = X_train[X_train.target_class==1]
    y_train_1 = y_train[X_train.target_class==1]
    
    dtrain_0 = lgbm.Dataset(X_train_0[cont_features], y_train_0)
    dvalid = lgbm.Dataset(X_val[cont_features], y_val)
    
    params = {"objective": "regression",
              "metric": "rmse",
              "verbosity": -1,
              "boosting_type": "gbdt",
              "feature_fraction":0.5,
              "num_leaves": 250,
              "lambda_l1":4,
              "lambda_l2":2,
              "learning_rate":0.01,
              'min_child_samples': 35,
              "bagging_fraction":0.75,
              "bagging_freq":1,
             }
    model_0 = lgbm.train(params,
                    dtrain_0,
                    valid_sets=[dtrain_0, dvalid],
                    verbose_eval=100,
                    num_boost_round=100000,
                    early_stopping_rounds=100
                )
    
    y_pred_0 = model_0.predict(X_val[cont_features])    
    score = np.sqrt(mean_squared_error(y_val, y_pred_0))
    print(f"RMSE Fold-{fold} : {score}")
    
    # Second model with only X_train having class = 1
    dtrain_1 = lgbm.Dataset(X_train_1[cont_features], y_train_1)
    dvalid = lgbm.Dataset(X_val[cont_features], y_val)
    params = {"objective": "regression",
              "metric": "rmse",
              "verbosity": -1,
              "boosting_type": "gbdt",
              "feature_fraction":0.5,
              "num_leaves": 250,
              "lambda_l1":4,
              "lambda_l2":2,
              "learning_rate":0.01,
              'min_child_samples': 35,
              "bagging_fraction":0.75,
              "bagging_freq":1,
             }
    model_1 = lgbm.train(params,
                    dtrain_1,
                    valid_sets=[dtrain_1, dvalid],
                    verbose_eval=100,
                    num_boost_round=100000,
                    early_stopping_rounds=100
                )
    
    y_pred_1 = model_1.predict(X_val[cont_features])    
    score = np.sqrt(mean_squared_error(y_val, y_pred_1))
    print(f"RMSE Fold-{fold} : {score}")
    
    # Then we are getting final prediction as following. Multiply first model output with the probability of being class 0, then multiply second model
    # output with the probability of being class 1 and then finally sum them up.
    oof_2[test_index] = y_pred_0*(1-X_val["class_preds"]) + y_pred_1*(X_val["class_preds"])
    score = np.sqrt(mean_squared_error(y_val,oof_2[test_index] ))
    print(f"RMSE Fold-Final-{fold} : {score}")
    

    score_list.append(score)
    test_preds_2.append(model_0.predict(test[cont_features])*(1-test["class_preds"])  + model_1.predict(test[cont_features])*(test["class_preds"]))
    fold+=1
    

np.mean(score_list)

In [ ]:
print(np.mean(score_list))
score_list

In [ ]:
train["2_preds"] = oof_2
test["2_preds"] = np.mean(test_preds_2,axis=0)
np.sqrt(mean_squared_error(train["target"],train["2_preds"] ))

## 3rd type of modeling: augmenting data with dae output.

Even though I failed with dae (given the 1st place's amazing solution :)), it added diverstiy to my final blending in the end. First I run a simple MLP model with a linear output and then run it for a couple epochs. Later on, I used the MLP output for augmenting the original training data during my CV.


In [ ]:
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras import Model
from  tensorflow.keras.regularizers import l2
import tensorflow as tf

In [ ]:
def get_DAE():
    # denoising autoencoder
    inputs = Input((14,))
    x = Dense(500, activation='relu')(inputs) # 1500 original
    x = Dense(500, activation='relu', name="feature")(x) # 1500 original
    x = Dense(500, activation='relu')(x) # 1500 original
    outputs = Dense(14, activation='relu')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')

    return model

In [ ]:
alldata = pd.concat([train[cont_features],test[cont_features]],axis=0)
print(alldata.shape)
autoencoder = get_DAE()
autoencoder.fit(alldata[cont_features], alldata[cont_features],
                    epochs=15,
                    batch_size=256,
                    shuffle=True
                    )

In [ ]:
# Getting denoised datasets.
test_denoised = test.copy()
test_denoised[cont_features] = autoencoder.predict(test_denoised[cont_features])
train_denoised = train.copy()
train_denoised[cont_features] = autoencoder.predict(train_denoised[cont_features])

In [ ]:
train_denoised.head()

In [ ]:
#X = X.abs()
y = train["target"]
kf = KFold(n_splits=5, shuffle=True, random_state=1)
oof_3 = np.zeros(len(train))
score_list = []
fold = 1
test_preds_3 = []
seed_list = [None,2,3]# Use more. Original list: [None,2,3,4,5]

for train_index, test_index in kf.split(train):
    X_train, X_val = train.iloc[train_index], train.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
    X_train_denoised, X_val_denoised = train_denoised.iloc[train_index], train_denoised.iloc[test_index]
    
    # Augmenting only training.
    print(X_train_denoised.shape)
    X_train_denoised = pd.concat([X_train_denoised,X_train],axis=0)
    y_train = pd.concat([y_train,y_train],axis=0)
    print(X_train_denoised.shape)
    
    y_pred_list = []
    
    for seed in seed_list:
        
        dtrain = lgbm.Dataset(X_train_denoised[cont_features], y_train)
        dvalid = lgbm.Dataset(X_val[cont_features], y_val)

        params = {"objective": "regression",
              "metric": "rmse",
              "verbosity": -1,
              "boosting_type": "gbdt",
              "feature_fraction":0.5,
              "num_leaves": 250,
              "lambda_l1":7,
              "lambda_l2":2,
              "learning_rate":0.01,
              'min_child_samples': 35,
              "bagging_fraction":0.75,
              "bagging_freq":1,
             }
        params["seed"] = seed
        model = lgbm.train(params,
                        dtrain,
                        valid_sets=[dtrain, dvalid],
                        verbose_eval=100,
                        num_boost_round=100000,
                        early_stopping_rounds=100
                    )
        
        
        dtrain = lgbm.Dataset(X_train_denoised[cont_features], y_train)
        dvalid = lgbm.Dataset(X_val[cont_features], y_val)
        
        params = {"objective": "regression",
                  "metric": "rmse",
                  "verbosity": -1,
                  "boosting_type": "gbdt",
                  "feature_fraction":0.5,
                  "num_leaves": 350,
                  "lambda_l1":7,
                  "lambda_l2":1,
                  "learning_rate":0.003,
                  'min_child_samples': 35,
                  "bagging_fraction":0.8,
                  "bagging_freq":1,
                 }
        
        params["seed"] = seed
        model = lgbm.train(params,
                            dtrain,
                            valid_sets=[dtrain, dvalid],
                            verbose_eval=100,
                            num_boost_round=100000,
                            early_stopping_rounds=200,
                           init_model = model
                        )
        
        

        y_pred_list.append(model.predict(X_val[cont_features]))
        print(np.sqrt(mean_squared_error(y_val, np.mean(y_pred_list,axis=0))))
        
        test_preds_3.append(model.predict(test[cont_features]))
        
    oof_3[test_index] = np.mean(y_pred_list,axis=0)    
    score = np.sqrt(mean_squared_error(y_val, oof_3[test_index]))
    score_list.append(score)
    print(f"RMSE Fold-{fold} : {score}")
    
    fold+=1

np.mean(score_list)

In [ ]:
print(np.mean(score_list))
score_list

In [ ]:
train["3_preds"] = oof_3
test["3_preds"] = np.mean(test_preds_3,axis=0)

I also have another modeling with full NN method. 

In [ ]:
train[["id","1_preds","2_preds","3_preds"]].to_csv("train_preds.csv",index=False)
test[["id","1_preds","2_preds","3_preds"]].to_csv("test_preds.csv",index=False)

Now I'll also read my NN predictions from another notebook and I'll apply stacking with all of my predictions.

You'll see that, stacking with LR is betten than dealing with manual weights.

In [ ]:
train["4_preds"] = pd.read_csv("../input/nn-with-categorical-embedding/nn_preds_train.csv")["4_preds"]
test["4_preds"] = pd.read_csv("../input/nn-with-categorical-embedding/nn_preds_test.csv")["4_preds"]

In [ ]:
print("1st model score: ",np.sqrt(mean_squared_error(train["target"], train["1_preds"])))
print("2nd model score: ",np.sqrt(mean_squared_error(train["target"], train["2_preds"])))
print("3rd model score: ",np.sqrt(mean_squared_error(train["target"], train["3_preds"])))
print("4th model score: ",np.sqrt(mean_squared_error(train["target"], train["4_preds"])))
print("Blending model score: ",np.sqrt(mean_squared_error(train["target"], train["4_preds"]*0.1+train["3_preds"]*0.3+train["2_preds"]*0.3+train["1_preds"]*0.3)))

In [ ]:
input_df = train[["1_preds","2_preds","3_preds","4_preds"]].copy()
sub_input_df = test[["1_preds","2_preds","3_preds","4_preds"]].copy()

**Important note:** To be able to have a safer evaluation and less leaking during stacking, it's always good practice to use same KFold scheme at every model, which I also did here.

In [ ]:
y = train["target"]
kf = KFold(n_splits=5, shuffle=True, random_state=1)
oof_stack = np.zeros(len(train))

score_list= []
fold = 1
test_preds_stack = []
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso

for train_index, test_index in kf.split(input_df):
    X_train, X_val = input_df.iloc[train_index], input_df.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]    
    
    reg = LinearRegression(fit_intercept=True).fit(X_train, y_train)
    y_stack = reg.predict(X_val)
    
    oof_stack[test_index] = y_stack*1
    score = np.sqrt(mean_squared_error(y_val, oof_stack[test_index]))
    score_list.append(score)
    
    test_preds_stack.append(reg.predict(sub_input_df.values).ravel())
    

np.sqrt(mean_squared_error(y, oof_stack))   

In [ ]:
print(np.mean(score_list))
score_list

### And now one more final step to squeeze more. Since linear regression model can only create linear relation between the features, I also wanted to create some non-linear interactions between input predictions. This method also gave some boost at 4t and 5th decimals, which was worth for this competition :)

In [ ]:
def final_features(df):
    combinations_list = [("1_preds","2_preds"),
                        ("1_preds","3_preds"),
                         ("1_preds","4_preds"),
                        ("2_preds","3_preds"),
                        ("2_preds","4_preds"),
                        ("3_preds","4_preds")]
    for i in range(len(combinations_list)):
        col1 = combinations_list[i][0]
        col2 = combinations_list[i][1]
        
        df.loc[:,f"{col1}_{col2}_multiply"] = df[col1] * df[col2]
        df.loc[:,f"{col1}_{col2}_divide"] = df[col1] / df[col2]
        
    combinations_list = [("1_preds","2_preds","3_preds"),
                        ("1_preds","2_preds","4_preds"),
                        ("1_preds","3_preds","4_preds"),
                        ("2_preds","3_preds","4_preds")]
    for i in range(len(combinations_list)):
        col1 = combinations_list[i][0]
        col2 = combinations_list[i][1]
        col3 = combinations_list[i][2]
        
        df.loc[:,f"{col1}_{col2}_{col3}_multiply"] = df[col1] * df[col2]* df[col3]        
    return df

input_df = train[["1_preds","2_preds","3_preds","4_preds"]].copy()
sub_input_df = test[["1_preds","2_preds","3_preds","4_preds"]].copy()
input_df = final_features(input_df)
sub_input_df = final_features(sub_input_df)
input_df.head()

In [ ]:
y = train["target"]
kf = KFold(n_splits=5, shuffle=True, random_state=1)
oof_stack = np.zeros(len(train))

score_list= []
fold = 1
test_preds_stack = []
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso

for train_index, test_index in kf.split(input_df):
    X_train, X_val = input_df.iloc[train_index], input_df.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
    
    reg = LinearRegression(fit_intercept=True).fit(X_train, y_train)
    y_stack = reg.predict(X_val)
    
    oof_stack[test_index] = y_stack*1
    score = np.sqrt(mean_squared_error(y_val, oof_stack[test_index]))
    score_list.append(score)
    
    test_preds_stack.append(reg.predict(sub_input_df.values).ravel())
    

np.sqrt(mean_squared_error(y, oof_stack))   

In [ ]:
print(np.mean(score_list))
score_list

By using mainly these 4 type of models and this stacking method at the end I reached to 0.694X CV score. If you want to get same score, you need to run samed models for seed_list = [None,2,3,4,5] and for the 2nd stage model, you need to generate more models with a threshold other than 8 to have more diverse models.

In [ ]:
train["stacked_preds"] = oof_stack
test["target"] = np.mean(test_preds_stack,axis=0)

In [ ]:
train[["id","stacked_preds"]].to_csv("tarin_stacked_oof.csv",index=False)
test[["id","target"]].to_csv("submission.csv",index=False)